In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite


In [2]:
# Load the Aspergillus niger ATCC1015 model
model = read_sbml_model('../model/Aspergillus_ATCC1015.xml')
model

'' is not a valid SBML 'SId'.


Name,
Memory address,0x024feccf0190
Number of metabolites,1818
Number of reactions,2320
Number of groups,0
Objective expression,1.0*DRAIN_Biomass - 1.0*DRAIN_Biomass_reverse_fcc30
Compartments,"Cytoplasm, Mitochondria, Extracellular, Peroxisome, Nucleus, Golgi, Vacuole"


In [3]:
# Find pathway and metabolites
for m in model.metabolites.query('Farnesyl', 'name'):
    print(m.id,'\n',m.name)

FPP 
 trans,trans-Farnesyl diphosphate


In [4]:
# Create all metabolites
Amorphadiene = Metabolite(id = 'amorpha', name = 'Amorphadiene', compartment = 'c')
Artemisinic_alcohol = Metabolite(id = 'artem_alcohol', name = 'Artemisinic alcohol', compartment = 'c')
Artemisinic_aldehyde = Metabolite(id = 'artem_aldehyde', name = 'Artemisinic aldehyde', compartment = 'c')
Artemisinic_acid_c = Metabolite(id = 'artem_acid_c', name = 'Artemisinic acid_c', compartment = 'c')
Artemisinic_acid_e = Metabolite(id = 'artem_acid_e', name = 'Artemisinic acid_e', compartment = 'e')


In [7]:
# Create all reactions
def add_reaction (meta1, meta2, name):
    new_reaction = Reaction(name)
    new_reaction.add_metabolites({
        meta1: -1,
        meta2: 1
    })
    model.add_reactions([new_reaction])
    print(new_reaction.build_reaction_string(), " added.")

add_reaction(model.metabolites.FPP, Amorphadiene, 'ADS_reaction')
add_reaction(Amorphadiene, Artemisinic_alcohol, 'CYP1/AV1CYPCPR1CYB5_reaction')
add_reaction(Artemisinic_alcohol, Artemisinic_aldehyde, 'ADH1_reaction')
add_reaction(Artemisinic_aldehyde, Artemisinic_acid_c, 'ALDH1_reaction')


#create exchange reaction for secretion
add_reaction(Artemisinic_acid_c, Artemisinic_acid_e, 'secretion')
model.add_boundary(model.metabolites.get_by_id("artem_acid_e"), type="exchange")

Ignoring reaction 'ADS_reaction' since it already exists.
Ignoring reaction 'CYP1/AV1CYPCPR1CYB5_reaction' since it already exists.
Ignoring reaction 'ADH1_reaction' since it already exists.
Ignoring reaction 'ALDH1_reaction' since it already exists.
Ignoring reaction 'secretion' since it already exists.


FPP --> amorpha  added.
amorpha --> artem_alcohol  added.
artem_alcohol --> artem_aldehyde  added.
artem_aldehyde --> artem_acid_c  added.
artem_acid_c --> artem_acid_e  added.


ValueError: Boundary reaction 'EX_artem_acid_e' already exists.

In [6]:
with model:
    model.objective = model.reactions.secretion
   # model.solver ="glpk"
    print(model.optimize().objective_value)
    

2.808134394341298
